# Delete API E2E Tests

In [1]:
import requests
import random
import string
import json

In [2]:
base_url = "http://localhost:8081"

In [3]:
resp = requests.get(base_url)
assert resp.status_code == 404
assert resp.json()["status"] == 404
assert resp.json()["error"] == "Not Found"

In [4]:
resp = requests.get("%s/api/docs/swagger.json"%base_url)
assert resp.status_code == 200
assert resp.json()["swagger"] == "2.0"
assert resp.json()["paths"] != None

In [5]:
swaggerJson = resp.json()
apiEndpoints = swaggerJson["paths"].keys()
assert len(apiEndpoints)==45

In [6]:
inviteUrls = [f for f in swaggerJson["paths"].keys() if "/invite" in f]
instituteInvite = [u for u in inviteUrls if "/Institute" in u][0]
teacherInvite = [v for v in inviteUrls if "/Teacher" in v][0]
studentInvite = [w for w in inviteUrls if "/Student" in w][0]
print(instituteInvite,teacherInvite,studentInvite)

/api/v1/Institute/invite /api/v1/Teacher/invite /api/v1/Student/invite


In [7]:
def entityForInstitute(userId):
    return  {
        "instituteName": "%s"%userId,
        "ContactDetails":{ "mobile":"%s"%userId,"email":"%s@example.com"%userId,
               "address":{"street":"abcde","state":"Karnataka","pincode":"560001"}},
        "category": "Primary",
        "schoolType": "Co-ed"
    }

In [8]:
def entityForTeacher(userId):
    return  {"personalDetails":{
         "userName":userId,
         "dob":"2020-12-12",
     },
      "IdentityDetails":{"type":"id","value":userId}, 
      "ContactDetails":{ "mobile":"%s"%userId,"email":"%s@example.com"%userId,
                       "address":{"street":"abc","state":"Karnataka","pincode":"560001"}}
     }

In [9]:
def getEntityPayload(userId,password):
    return {
      "client_id": "registry-frontend",
      "username": "%s"%userId,
      "password": "%s"%password,
      "grant_type": "password"
    }

In [10]:
defaultPassowrd = "abcd@123"

## Invite 1st Institute

In [11]:
institute1Id =str(random.randint(1e10,1e11))
institute1Id

'29327853293'

In [12]:
resp = requests.post("%s%s"%(base_url, instituteInvite), json=entityForInstitute(institute1Id)
)
print(resp.content)
assert resp.status_code == 200
print(resp.json())
assert(resp.json()["id"]=="sunbird-rc.registry.invite")
assert resp.json()["params"]["status"] == "SUCCESSFUL"
entity_name=next(iter(resp.json()["result"].keys()))
assert "Institute"==entity_name
institute1OsId = resp.json()["result"][entity_name]["osid"]
institute1OsId

b'{"id":"sunbird-rc.registry.invite","ver":"1.0","ets":1646139005822,"params":{"resmsgid":"","msgid":"7a476473-b1ad-48d2-8a1b-fbf773b1d3ca","err":"","status":"SUCCESSFUL","errmsg":""},"responseCode":"OK","result":{"Institute":{"osid":"1-abe9ee39-8606-4425-98cb-60942eaa717a"}}}'
{'id': 'sunbird-rc.registry.invite', 'ver': '1.0', 'ets': 1646139005822, 'params': {'resmsgid': '', 'msgid': '7a476473-b1ad-48d2-8a1b-fbf773b1d3ca', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'Institute': {'osid': '1-abe9ee39-8606-4425-98cb-60942eaa717a'}}}


'1-abe9ee39-8606-4425-98cb-60942eaa717a'

## Invite Teacher 1 using 1st Institute

### Get Institute 1 token

In [16]:
headers = {
    'content-type': 'application/x-www-form-urlencoded',
}
data = getEntityPayload(institute1Id,defaultPassowrd)
response = requests.post('http://keycloak:8080/auth/realms/sunbird-rc/protocol/openid-connect/token', headers=headers, data=data)
institute1Token = response.json()["access_token"]
institute1Token

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJZcGNvVHQ4bTBveXZkbjY4dU9icE0zX1REZWwzbXFhWXA5d19hX19PY3g0In0.eyJleHAiOjE2NDYxMzk2NDEsImlhdCI6MTY0NjEzOTA0MSwianRpIjoiMzNjN2ZmYTUtMjVhMy00MjRmLWEwODItYjk2YzdlYmRmM2FkIiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiI3ZmQ5OTRkYi0wN2JjLTQwOGQtYTUzYS0wNmEzYzc5YmQ1NTIiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiIwYWQ0Y2Q4YS0yMTUyLTQyOTYtYWEzZi0wYTliOTdhMmQwOWEiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbImVkdS1hZG1pbiIsIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidml

In [17]:
teacher1Id =str(random.randint(1e10,1e11))
teacher1Id

'14865764048'

### Invite Teacher

In [18]:
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%institute1Token,
}
resp = requests.post("%s%s"%(base_url, teacherInvite), json=entityForTeacher(teacher1Id),headers= headers)
print(resp.content)
assert resp.status_code == 200
print(resp.json())
assert(resp.json()["id"]=="sunbird-rc.registry.invite")
assert resp.json()["params"]["status"] == "SUCCESSFUL"
entity_name=next(iter(resp.json()["result"].keys()))
assert "Teacher"==entity_name
teacher1OsId = resp.json()["result"][entity_name]["osid"]
teacher1OsId

b'{"id":"sunbird-rc.registry.invite","ver":"1.0","ets":1646139044239,"params":{"resmsgid":"","msgid":"418d6742-b754-4989-b7a7-cf613b06681b","err":"","status":"SUCCESSFUL","errmsg":""},"responseCode":"OK","result":{"Teacher":{"osid":"1-84cf1bf1-5176-41af-bfbc-b02b7e992e98"}}}'
{'id': 'sunbird-rc.registry.invite', 'ver': '1.0', 'ets': 1646139044239, 'params': {'resmsgid': '', 'msgid': '418d6742-b754-4989-b7a7-cf613b06681b', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'Teacher': {'osid': '1-84cf1bf1-5176-41af-bfbc-b02b7e992e98'}}}


'1-84cf1bf1-5176-41af-bfbc-b02b7e992e98'

## Invite 2nd Institute

In [19]:
instituteId2 =str(random.randint(1e10,1e11))
instituteId2

'90508239224'

In [20]:
resp = requests.post("%s%s"%(base_url, instituteInvite), json=entityForInstitute(instituteId2)
)
print(resp.content)
assert resp.status_code == 200
print(resp.json())
assert(resp.json()["id"]=="sunbird-rc.registry.invite")
assert resp.json()["params"]["status"] == "SUCCESSFUL"
entity_name=next(iter(resp.json()["result"].keys()))
assert "Institute"==entity_name
institute2OsId = resp.json()["result"][entity_name]["osid"]
institute2OsId

b'{"id":"sunbird-rc.registry.invite","ver":"1.0","ets":1646139055487,"params":{"resmsgid":"","msgid":"21a20e02-706e-483f-9bc8-31c86d3416f3","err":"","status":"SUCCESSFUL","errmsg":""},"responseCode":"OK","result":{"Institute":{"osid":"1-66030026-ee4e-43a1-8139-2bc7b706c451"}}}'
{'id': 'sunbird-rc.registry.invite', 'ver': '1.0', 'ets': 1646139055487, 'params': {'resmsgid': '', 'msgid': '21a20e02-706e-483f-9bc8-31c86d3416f3', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'Institute': {'osid': '1-66030026-ee4e-43a1-8139-2bc7b706c451'}}}


'1-66030026-ee4e-43a1-8139-2bc7b706c451'

## Create Teacher 2 with Institute 2

### Get Institute 2 token

In [34]:
headers = {
    'content-type': 'application/x-www-form-urlencoded',
}
data = getEntityPayload(instituteId2,defaultPassowrd)
response = requests.post('http://keycloak:8080/auth/realms/sunbird-rc/protocol/openid-connect/token', headers=headers, data=data)
institute2Token = response.json()["access_token"]
institute2Token

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJZcGNvVHQ4bTBveXZkbjY4dU9icE0zX1REZWwzbXFhWXA5d19hX19PY3g0In0.eyJleHAiOjE2NDYxNDAyNDUsImlhdCI6MTY0NjEzOTY0NSwianRpIjoiNmU5Nzk1M2QtZTdhNy00NDUwLWJiYjUtMzVkZDJmNmM5NmU0IiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiJjZjY3YjVkYy04NjZiLTQ0NTQtODM0Yy1lYjFiNDlmNGY5MDMiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiIzZTgyMzQ4MC00MjVlLTQ5OGEtYTI5ZS1lODVjODQwMThiMmUiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbImVkdS1hZG1pbiIsIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidml

### Create Managed Entity Teacher

In [22]:
entityToCreate = "Teacher"

In [25]:
teacherId2 =str(random.randint(1e10,1e11))
teacherId2

'79382525368'

In [23]:
print("%s%s%s"%(base_url,"/api/v1/",entityToCreate))

http://localhost:8081/api/v1/Teacher


In [33]:
# headers = {
#     'content-type': 'application/json',
#     'authorization': 'bearer %s'%institute2Token,
# }
# resp = requests.post("%s%s%s"%(base_url,"/api/v1/",entityToCreate), json=entityForTeacher(teacherId2),headers= headers)
# print(resp.content)
# assert resp.status_code == 200
# print(resp.json())
# assert(resp.json()["id"]=="sunbird-rc.registry.invite")
# assert resp.json()["params"]["status"] == "SUCCESSFUL"
# entity_name=next(iter(resp.json()["result"].keys()))
# assert "Teacher"==entity_name
# teacher2OsId = resp.json()["result"][entity_name]["osid"]
# teacher2OsId

## Delete Teacher 2 with Institute 2

In [22]:
entityToDelete = "Teacher"

In [ ]:
# headers = {
#     'content-type': 'application/json',
#     'authorization': 'bearer %s'%institute2Token,
# }
# resp = requests.delete("%s/api/v1/%s/%s"%(base_url,entityToDelete,teacher2OsId),headers= headers)
# resp.json()

## Delete Teacher 1 with Institute 1

In [23]:
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%institute1Token,
}
resp = requests.delete("%s/api/v1/%s/%s"%(base_url,"Teacher",teacher1OsId),headers= headers)
resp.json()
assert resp.status_code== 200

{'id': 'sunbird-rc.registry.delete',
 'ver': '1.0',
 'ets': 1646139101603,
 'params': {'resmsgid': '',
  'msgid': 'b38a55f7-faf8-494e-a1ed-935a5ecad9b4',
  'err': '',
  'status': 'SUCCESSFUL',
  'errmsg': ''},
 'responseCode': 'OK'}

### Should get 401 bad request when try GET the deleted Teacher token

In [29]:
headers = {
    'content-type': 'application/x-www-form-urlencoded',
}
data = getEntityPayload(teacher1OsId,defaultPassowrd)
response = requests.post('http://keycloak:8080/auth/realms/sunbird-rc/protocol/openid-connect/token', headers=headers, data=data)
response.json()
assert response.status_code == 401
assert response.json()['error_description'] == 'Invalid user credentials'

## Delete Institute 2 itself

In [37]:
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%institute2Token,
}
resp = requests.delete("%s/api/v1/%s/%s"%(base_url,"Institute",institute2OsId),headers= headers)
resp.json()

{'id': 'sunbird-rc.registry.update',
 'ver': '1.0',
 'ets': 1646139697937,
 'params': {'resmsgid': '',
  'msgid': 'a0921a7f-e021-4204-95ea-2cefecb27aa7',
  'err': '',
  'status': 'UNSUCCESSFUL',
  'errmsg': 'entity status is inactive'},
 'responseCode': 'OK'}

### Should get 401 bad request when try GET the deleted Institute token

In [38]:
headers = {
    'content-type': 'application/x-www-form-urlencoded',
}
data = getEntityPayload(institute2OsId,defaultPassowrd)
response = requests.post('http://keycloak:8080/auth/realms/sunbird-rc/protocol/openid-connect/token', headers=headers, data=data)
assert response.status_code == 401
assert response.json()['error_description'] == 'Invalid user credentials'